In [5]:
import heapq, random

class PriorityQueue:
    """
      Implements a priority queue data structure. Each inserted item
      has a priority associated with it and the client is usually interested
      in quick retrieval of the lowest-priority item in the queue. This
      data structure allows O(1) access to the lowest-priority item.
    """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        # If item already in priority queue with equal or lower priority, do nothing.
        # If item not in priority queue, do the same thing as self.push.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)

In [18]:
class Person:
    def __init__(self, height: int) -> None:
        """
        height: height in inches rounded to nearest whole number
        """
        self.height = height
        self.wingspan = height * 1.06
        self.reach = height * 1.35

In [19]:
class Hold:
    def __init__(self, x: int, y: int, dif: float, width: float, height: float, angle: int):
        #Coords = top left corner
        self.x = x
        self.y = y
        self.dif = dif
        self.width = width
        self.height = height
        self.angle = angle
    def getCenter(self):
        return (self.coords[0] + self.width/2, self.coords[1] + self.height/2)

In [20]:
from enum import Enum

class LimbName(Enum):
    LEFT_HAND = "Left Hand"
    RIGHT_HAND = "Right Hand"
    LEFT_LEG = "Left Leg"
    RIGHT_LEG = "Right Leg"


In [21]:
class Limb:
    def __init(self, name: LimbName, strength: int, flexibility: int, hold: Hold):
        self.name, self.strength, self.flexibility, self.hold = name, strength, flexibility, hold

In [22]:
class Route:
    def __init__(self, holds: Hold, start1: Hold, start2: Hold, finish: Hold):
        self.holds = [(0, 1), (1, 1), (1, 3), (4, 3), (6, 7), (10, 11), (14, 12), (15, 15)]
        self.start_hold1 = (0, 1)
        self.start_hold2 = (1, 1)
        self.finish_hold = (15, 15)

In [23]:
import math


class State:
    def __init__(self, lf: Limb, rf: Limb, lh: Limb, rh: Limb, person: Person):
        self.lf, self.rf, self.lh, self.rh = lf, rf, lh, rh
        self.person = person
        self.wall_height_inches = 180 # Represents real height of wall <- need to measure
        self.wall_height_pixels = 1920 #placeholder <- Need to extract from image

    # Not sure where to put this
    def inches_to_pixels(self, inches: int):
        return (self.wall_height_pixels / self.wall_height_inches) * inches
    
def move_difficulty(state: State, limb: Limb, next_hold: Hold):
    distance = math.sqrt(((limb.hold.x - next_hold.x) ** 2) + ((limb.hold.y - next_hold.y) ** 2))
    distance_diff = 0.5 * distance

    lh, rh, lf, rf, person = state.lh, state.rh, state.lf, state.rf, state.person
    new_state_limbs = [lh, rh, lf, rf]
    for new_state_limb in new_state_limbs:
        if new_state_limb.name == limb.name:
            new_state_limb.hold = None
    new_state = State(lh=lh, rh=rh, lf=lf, rf=rf, person=person)
    state_without_limb_difficulty = 0.5 * state_difficulty(new_state)
    move_diff = distance_diff + state_without_limb_difficulty
    
    return move_diff

def state_difficulty(state: State):
    average_hands_x = (state.lh.hold.x + state.rh.hold.x) / 2 if state.lh.hold != None and state.rh.hold != None else state.lh.hold.x if state.rh.hold == None else state.rh.hold.x
    average_legs_x = (state.lf.hold.x + state.rf.hold.x) / 2 if state.lf.hold != None and state.rf.hold != None else state.lf.hold.x if state.rf.hold == None else state.rf.hold.x
    average_hands_y = (state.lh.hold.y + state.rh.hold.y) / 2 if state.lh.hold != None and state.rh.hold != None else state.lh.hold.y if state.rh.hold == None else state.rh.hold.y
    average_legs_y = (state.lf.hold.y + state.rf.hold.y) / 2 if state.lh.hold != None and state.rh.hold != None else state.lf.hold.y if state.rh.hold == None else state.rf.hold.y

    hands_difference_x = abs(state.lh.x - state.rh.x) if state.lh.hold != None and state.rh.hold != None else 0
    hands_difference_y = abs(state.lh.y - state.rh.y) if state.lh.hold != None and state.rh.hold != None else 0

    legs_difference_x = abs(state.lf.x - state.rf.x) if state.lf.hold != None and state.rf.hold != None else 0
    legs_difference_y = abs(state.lf.y - state.rf.y) if state.lf.hold != None and state.rf.hold != None else 0

    diff = 0
    center_diff = 0.5*abs(average_hands_x - average_legs_x)

    target_distance = state.inches_to_pixels(state.inches_to_pixels(state.person.height * 0.8))
    scaling_factor = 0.1
    distance_diff = abs(average_legs_y - average_hands_y) + target_distance

    raw_difficulty_score = (scaling_factor * (distance_diff ** 2))

    # Scales the difficulty score linearly between 1 and 10
    scaled_difficulty_score = 1 + 9 * ((raw_difficulty_score) / (1 + scaling_factor * target_distance ** 2))

    limb_strength_diff = 0
    orientation_diff = 0
    for limb in [state.lh, state.rh, state.lf, state.rf]:
        if limb.hold != None:
            limb_strength_diff += limb.hold.diff / limb.strength
            if limb.name == LimbName.LEFT_HAND:
                if 315 >= limb.hold.orientation >= 270:
                    orientation_diff += 2
                elif 90 >= limb.hold.orientation or limb.hold.orientation > 315:
                    orientation_diff += 1
                elif 180 >= limb.hold.orientation > 90:
                    orientation_diff += 2.5
                else:
                    orientation_diff += 3
            if limb.name == LimbName.RIGHT_HAND:
                if 90 >= limb.hold.orientation >= 45:
                    orientation_diff += 2
                elif 45 >= limb.hold.orientation or limb.hold.orientation > 270:
                    orientation_diff += 1
                elif 270 >= limb.hold.orientation > 180:
                    orientation_diff += 2.5
                else:
                    orientation_diff += 3
            if limb.name in [LimbName.LEFT_LEG, LimbName.RIGHT_LEG]:
                if 90 <= limb.hold.orientation <= 270:
                    orientation_diff += 2
        else:
            limb_strength_diff += 6
    if hands_difference_y > self.inches_to_pixels(self.inches_to_pixels(0.6 * self.person.wingspan)): 
        diff += 0.5 * hands_difference_y 
    if hands_difference_x > self.inches_to_pixels(self.inches_to_pixels(0.8 * self.person.wingspan)):
        diff += 0.5 * hands_difference_x
    if legs_difference_y > self.inches_to_pixels(self.inches_to_pixels(0.25 * self.person.height)):
        diff += 0.5 * legs_difference_y
    if legs_difference_x > self.inches_to_pixels(self.inches_to_pixels(0.6 * self.person.wingspan)):
        diff += 0.5 * legs_difference_x
    diff += center_diff + scaled_difficulty_score + orientation_diff + limb_strength_diff
    return diff

In [24]:
import numpy as np
class aStar:
    WALL_HEIGHT = 180 #image.height -> divide that by height of wall in inches to get conversion
    def __init__(self, route):
        self.route = route
#         self.holds = route.holds
        self.explored = []
        self.reach = 96 #from foot
        self.limbs = [Limbs.LEFT_HAND, Limbs.RIGHT_HAND, Limbs.LEFT_LEG, Limbs.RIGHT_LEG]        
    
    def getSuccessors(self, cur_hold, limb, state):
        successors = []

        for hold in self.route.holds:
            if (self.get_distance(hold, cur_hold) < self.reach and 
            self.eval_difficulty(limb, hold)[0] < 10 and hold not in self.explored):
                successors.append((hold, self.eval_difficulty(limb, cur_hold, hold, state)[1], 
                                   self.eval_difficulty(limb, cur_hold, hold, state)[0]))
#         print(successors[0][1])
        return successors
    
    def get_distance(self, hold, state):
        return np.sqrt((state[0] - hold[0]) ** 2 + (state[1] - hold[1]) ** 2)
    
    
        #return (1, "move")
    
    #def convert_pixel_to_inch(self):
        
    
#     def eval_action(self, state):
#         dict2 = {}
#         for limb in limbs:
#             for hold in self.route.holds:
#                 If dict1[limb].contains(hold):
#                     pass
#                 elif distance(limb.pos, hold.pos) >some number:
#                     pass
#                 else:
#                     difficulty = evaluation(action)
#                     dict2[(limb,hold)]= difficulty
#         return min(dict2)

    
    def uniformCostSearch(self):
        state = [None, None, None, None]
        l_dif = self.eval_difficulty(self.limbs[0], self.route.start_hold1)[0]
        r_dif = self.eval_difficulty(self.limbs[1], self.route.start_hold1)[0]
        frontier = PriorityQueue()

        if l_dif < r_dif:
            state = [self.route.start_hold1,self.route.start_hold2, None, None]
            cur_state1 = [state[0], [], l_dif]
            cur_state2 = [state[1], [], r_dif]
            frontier.push(cur_state1, l_dif)
            frontier.push(cur_state2, r_dif)
        else:
            state = [self.route.start_hold2, self.route.start_hold1, None, None]
            cur_state1 = [state[0], [], l_dif]
            cur_state2 = [state[1], [], r_dif]
            frontier.push(cur_state2, r_dif)
            frontier.push(cur_state1, l_dif)

        index = 2
        explored = {0:[], 1:[], 2:[], 3:[]}
        while not frontier.isEmpty():
            current_limb = self.limbs[index]
            cur_hold = frontier.pop()
            explored[index].append(cur_hold[0])
            state[index] = cur_hold[0]
            if cur_hold[0] == self.route.finish_hold:
                print(cur_hold[1])
                return cur_hold[1]
            for hold in self.getSuccessors(cur_hold[0], current_limb, state):
                next_cost = cur_hold[2] + hold[2]
                next_hold = [hold[0], cur_hold[1] + [hold[1]], next_cost]
                if hold[0] not in explored[index]: 
                    add_cond = True
                    for item in frontier.heap:
                        if item[2][0] == hold[0]:
                            if next_cost < item[2][2]: 
                                item[2][1] = next_hold[1]
                                frontier.update(item[2], next_cost)
                            add_cond = False
                            break
                    if add_cond:
                        frontier.push(next_hold, next_cost)
                        state[index] = next_hold
            index += 1 
            index = index % len(self.limbs)
#             print(index)


In [25]:
route = Route()
a_star = aStar(route)

print(a_star.uniformCostSearch())


TypeError: Route.__init__() missing 4 required positional arguments: 'holds', 'start1', 'start2', and 'finish'